In [1]:
import hydraseq as hds

In [2]:
hs = hds.Hydraseq("_")

In [3]:
faux = [[0, 2, ['alpha']], [2, 5, ['beta','zeta']], [2, 3, ['gamma']], [3, 5, ['niarfe']],[5, 7, ['delta']]]

In [4]:
def generate_tree(lst_nods, debug=False):
    print("generate_tree")
    seq = hds.Hydraseq("_")
    seq.insert("0")
    zero = seq.columns["0"][0]
    for idx, nod in enumerate(lst_nods):
        node = hds.Node(nod[2])
        for nd in seq.columns[str(nod[0])]:
            print("nd from ", str(nod[0]))
            nd.nexts.append(node)
            node.lasts.append(nd)
            if debug: print("A inserting a node at {}".format(str(nod[1])))
            seq.columns[str(nod[1])].append(node)
            break;
        else:
            zero.nexts.append(node)
            node.lasts.append(zero)
            if debug: print("B inserting a node at {}".format(str(nod[1])))
            seq.columns[str(nod[1])].append(node)
        
    return seq
    

In [5]:
s = generate_tree(faux)

generate_tree
nd from  0
nd from  2
nd from  2
nd from  3
nd from  5


In [6]:
s.columns

defaultdict(list,
            {'0': [<node: 0,(*) 0>],
             '2': [<node: ['alpha'],(*) 0 ['alpha']>],
             '3': [<node: ['gamma'],(*) 0 ['alpha'] ['gamma']>],
             '5': [<node: ['beta', 'zeta'],(*) 0 ['alpha'] ['beta', 'zeta']>,
              <node: ['niarfe'],(*) 0 ['alpha'] ['gamma'] ['niarfe']>],
             '7': [<node: ['delta'],(*) 0 ['alpha'] ['beta', 'zeta'] ['delta']>]})

In [7]:
def flatten_tree(seq):
    print("flatten_tree")
    sequences = []
    for node_tup in [(key, node) for key, lst_nodes in seq.columns.items() for node in lst_nodes if not node.nexts]:
        key, node = node_tup
        print(key, node.get_sequence())
        outcome = " ".join(node.get_sequence().split()[2:])
        found_list = list(eval(outcome.replace("] [", "], [")))
        sequences.append(found_list)
    return sequences
       
print(flatten_tree(s))

flatten_tree
5 (*) 0 ['alpha'] ['gamma'] ['niarfe']
7 (*) 0 ['alpha'] ['beta', 'zeta'] ['delta']
[[['alpha'], ['gamma'], ['niarfe']], [['alpha'], ['beta', 'zeta'], ['delta']]]


## Problem, generate tree inserts 6 nodes, flatten tree identifies 5..

In [8]:

from hydraseq import Hydraseq, run_convolutions


hdq1 = Hydraseq('0_')
for pattern in [
    "the 0_A",
    "quick 0_ADJ",
    "brown 0_ADJ",
    "fox 0_N",
    "fox 0_V",
    "jumped 0_V",
    "over 0_PR",
    "lazy 0_ADJ",
    "dog 0_N"
]:
    hdq1.insert(pattern)

hdq2 = Hydraseq('_')
for pattern in [
    "0_N _NP_",
    "0_ADJ 0_N _NP_",
    "0_V _VP_",
    "0_ADV 0_V _VP_",
    "0_A 0_N _NP_",
    "0_A 0_ADJ 0_ADJ 0_N _NP_",

]:
    hdq2.insert(pattern)

hdq3 = Hydraseq('3_')
for pattern in [
    "_NP_ _VP_ 3_BINGO"
]:
    hdq3.insert(pattern)

sentence = "the quick brown fox jumped over the lazy dog"

hdq0 = Hydraseq('0_')
for word in sentence.split():
    hdq0.insert("{} 0_{}".format(word,word))
#print(run_convolutions(sentence.split(), hdq0))
result = run_convolutions(sentence.split(), hdq1)
expected = [[0, 1, ['0_A']], [1, 2, ['0_ADJ']], [2, 3, ['0_ADJ']], [3, 4, ['0_N', '0_V']], [4, 5, ['0_V']], [5, 6, ['0_PR']], [6, 7, ['0_A']], [7, 8, ['0_ADJ']], [8, 9, ['0_N']]]
assert result == expected
s = generate_tree(result)
encodeds = flatten_tree(s)
assert encodeds == [[['0_A'], ['0_ADJ'], ['0_ADJ'], ['0_N', '0_V'], ['0_V'], ['0_PR'], ['0_A'], ['0_ADJ'], ['0_N']]]

results = run_convolutions(encodeds[0], hdq2)
expected = [[0, 4, ['_NP_']], [2, 4, ['_NP_']], [3, 4, ['_NP_', '_VP_']], [4, 5, ['_VP_']], [7, 9, ['_NP_']], [8, 9, ['_NP_']]]
assert results == expected
s = generate_tree(results, debug=True)
encodeds = flatten_tree(s)
print(encodeds)
assert encodeds == [['_NP_'], ['_NP_', '_VP_'], [['_NP_'], ['_VP_']], ['_NP_'], ['_NP_']]
print(results)
for key, nodes in s.columns.items():
    print(key, nodes)


expected = [[0,2, ['3_BINGO']]]
results = run_convolutions(encodeds, hdq3)
print(results)
assert results == expected

generate_tree
nd from  0
nd from  1
nd from  2
nd from  3
nd from  4
nd from  5
nd from  6
nd from  7
nd from  8
flatten_tree
9 (*) 0 ['0_A'] ['0_ADJ'] ['0_ADJ'] ['0_N', '0_V'] ['0_V'] ['0_PR'] ['0_A'] ['0_ADJ'] ['0_N']
generate_tree
nd from  0
A inserting a node at 4
B inserting a node at 4
B inserting a node at 4
nd from  4
A inserting a node at 5
B inserting a node at 9
B inserting a node at 9
flatten_tree
4 (*) 0 ['_NP_']
4 (*) 0 ['_NP_', '_VP_']
5 (*) 0 ['_NP_'] ['_VP_']
9 (*) 0 ['_NP_']
9 (*) 0 ['_NP_']
[['_NP_'], ['_NP_', '_VP_'], [['_NP_'], ['_VP_']], ['_NP_'], ['_NP_']]
[[0, 4, ['_NP_']], [2, 4, ['_NP_']], [3, 4, ['_NP_', '_VP_']], [4, 5, ['_VP_']], [7, 9, ['_NP_']], [8, 9, ['_NP_']]]
0 [<node: 0,(*) 0>]
4 [<node: ['_NP_'],(*) 0 ['_NP_']>, <node: ['_NP_'],(*) 0 ['_NP_']>, <node: ['_NP_', '_VP_'],(*) 0 ['_NP_', '_VP_']>]
2 []
3 []
5 [<node: ['_VP_'],(*) 0 ['_NP_'] ['_VP_']>]
7 []
9 [<node: ['_NP_'],(*) 0 ['_NP_']>, <node: ['_NP_'],(*) 0 ['_NP_']>]
8 []
[[0, 2, ['3_BINGO']]]


In [9]:
hdq2.look_ahead([['0_A'], ['0_ADJ'], ['0_ADJ'], ['0_N', '0_V']]).get_next_values()

['_NP_']

In [10]:
hydra = Hydraseq(1, hdq2)

In [11]:
hydra.hit(['0_N', '0_V'], is_learning=False).get_next_values()

['_NP_', '_VP_']